In [1]:
import os
os.chdir('../')
current_path = os.getcwd()


In [2]:
import sympycalcs.helpers as sh
import sympy as sp 

import sympy.physics.units as unit
from sympy.abc import *
sp.init_printing(use_latex='mathjax', latex_mode='equation')


# Beispiel: Rayleigh-Quotient an einem Siebenmassenschwinger beim Ersatzkraftverfahren

{{< pagebreak >}}

## Aufgabenstellung

Das Beispiel ist aus @Dupraz2004 Seite 77 entnommen. Die Lastermittlung wird übernommen. Es wird vom vereinfachten Modell in @fig-ekv_6_modell ausgegangen. Der Fokus liegt auf der Bestimmung der Grundfrequenz.

![Vereinfachtes Modell für einen Mehrmassenschwinger](bilder/aufgabe_ekv_sieben_biege.svg){#fig-ekv_6_modell}

Gesucht:

- Erstelle die Nachgiebigkeitsmatrix (Verwende das Bildungsgesetz)
- Grundschwingzeit

Gegeben:




In [3]:
H_i, m_i,  I, I_x, I_y, f, f_y,f_x, T_x, T_y= sp.symbols('H_i, m_i  I  I_x I_y f f_y f_x T_x T_y')
n = 6 # n-Massenschwinger
F = sp.MatrixSymbol('F', n,1)
u = sp.MatrixSymbol('u', n,1)
u_y = sp.MatrixSymbol('u_y', n,1)
u_x = sp.MatrixSymbol('u_x', n,1)
f_hat = sp.MatrixSymbol('\hat{f}', n,n)
M = sp.MatrixSymbol('M', n,1)


In [4]:
params={
    E: 27*10**9*unit.newton/ unit.meter**2,
    H_i: 3.105*unit.meter,
    m_i: 1278*10**3*unit.newton*unit.second**2 /unit.meter,
    I_x: 14.89*unit.meter**4,
    I_y: 28.27*unit.meter**4,
}

params_plot = sh.params_value(params)

sh.dict_to_table(params)
sh.dict_to_table(params_plot)


|  Parameter  | ​  |
|---|---|
| $E = \frac{27000000000 \text{N}}{\text{m}^{2}}$ | $H_{i} = 3.105 \text{m}$ |
| $I_{x} = 14.89 \text{m}^{4}$ | $I_{y} = 28.27 \text{m}^{4}$ |
| $m_{i} = \frac{1278000 \text{N} \text{s}^{2}}{\text{m}}$ | ​  |


|  Parameter  | ​  |
|---|---|
| $E = 27000000000.0$ | $H_{i} = 3.105$ |
| $I_{x} = 14.89$ | $I_{y} = 28.27$ |
| $m_{i} = 1278000.0$ | ​  |


{{< pagebreak >}}

## Musterlösung

### Rayleigh-Quotient

In [5]:
def nachgiebigkeitsmatrix_nach_bildungsgesetz(n):
    """
    Erstellt die Nachgiebigkeitsmatrix nach dem Bildungsgesetz. Dieses ist nur zulässig für gleichmässige Stockwerkshöhen, sowie gleichbleibenden Stockwerkssteifigkeiten.
    Dies muss abschliessend mit h^3/(6*E*I) mutlipliziert werden.
    """
    from sympy import symbols, zeros

    
    def bildungsgesetz(i,j):
        return j**2*(3*i-j)

    matrix = sp.zeros(n, n)
    matrix_symbols =sp.zeros(n,n)
    matrix_unsymm = sp.zeros(n, n)
    
    for i in range(1,n+1):
        for j in range(1,n+1):
            if i >=j:
                matrix[i-1,j-1] = bildungsgesetz(i,j)
                matrix_symbols[i-1,j-1] = sp.Symbol(f'f_{i,j}')
                matrix_unsymm[i-1,j-1] = bildungsgesetz(i,j)

            if j>i:
                matrix[i-1, j-1] = bildungsgesetz(j,i)
                matrix_symbols[i-1,j-1] = sp.Symbol(f'f_{i,j}')

    
    return matrix, matrix_symbols, matrix_unsymm
            


In [6]:
eq_f_hat = sp.Eq(f_hat, sp.UnevaluatedExpr(H_i**3/(6*E*I))*nachgiebigkeitsmatrix_nach_bildungsgesetz(n)[0])

eq_F = sp.Eq(F,sp.Matrix(list(range(1, n+1))))
eq_u = sp.Eq(u,f_hat * F)


display(eq_f_hat, eq_F, eq_u)

          ⎡     3         3          3          3          3          3  ⎤
          ⎢   Hᵢ        Hᵢ         Hᵢ         Hᵢ         Hᵢ         Hᵢ   ⎥
          ⎢2⋅─────   5⋅─────    8⋅─────   11⋅─────   14⋅─────   17⋅───── ⎥
          ⎢  6⋅E⋅I     6⋅E⋅I      6⋅E⋅I      6⋅E⋅I      6⋅E⋅I      6⋅E⋅I ⎥
          ⎢                                                              ⎥
          ⎢     3          3         3          3          3          3  ⎥
          ⎢   Hᵢ         Hᵢ        Hᵢ         Hᵢ         Hᵢ         Hᵢ   ⎥
          ⎢5⋅─────   16⋅─────  28⋅─────   40⋅─────   52⋅─────   64⋅───── ⎥
          ⎢  6⋅E⋅I      6⋅E⋅I     6⋅E⋅I      6⋅E⋅I      6⋅E⋅I      6⋅E⋅I ⎥
          ⎢                                                              ⎥
          ⎢     3          3         3          3           3          3 ⎥
          ⎢   Hᵢ         Hᵢ        Hᵢ         Hᵢ          Hᵢ         Hᵢ  ⎥
          ⎢8⋅─────   28⋅─────  54⋅─────   81⋅─────   108⋅─────  135⋅─────⎥
          ⎢  6⋅E⋅I      6

    ⎡1⎤
    ⎢ ⎥
    ⎢2⎥
    ⎢ ⎥
    ⎢3⎥
F = ⎢ ⎥
    ⎢4⎥
    ⎢ ⎥
    ⎢5⎥
    ⎢ ⎥
    ⎣6⎦

u = \hat{f}⋅F

In [7]:
eq_M = sp.Eq(M,sp.ones(n,1)*m_i)


In [8]:
eq_f = sp.Eq(f,sp.Mul(sp.UnevaluatedExpr(1 / (2*sp.pi)),sp.sqrt(sp.Sum(sp.HadamardProduct(F, u)[i,0], (i, 0, F.shape[0]-1)) / sp.Sum(sp.HadamardProduct(M,u.applyfunc(lambda x:x**2))[i,0],(i,0,M.shape[0]-1))), evaluate=False))

eq_T = sp.Eq(T,1 / f)




Mittels dem Rayleigh-Quotient für das vereinfachte Modell lässt sich die Grundfrequenz direkt bestimmen.

In [9]:
display(eq_f, eq_F, eq_M, eq_T)

                          ____________________________
                         ╱   5                        
                        ╱   ___                       
                       ╱    ╲                         
                      ╱      ╲                        
                     ╱       ╱   (F)[i, 0]⋅(u)[i, 0]  
                    ╱       ╱                         
                   ╱        ‾‾‾                       
     1            ╱        i = 0                      
f = ───⋅         ╱         ────────────────────────── 
    2⋅π         ╱            5                        
               ╱            ___                       
              ╱             ╲                         
             ╱               ╲                2       
            ╱                ╱   (M)[i, 0]⋅(u) [i, 0] 
           ╱                ╱                         
          ╱                 ‾‾‾                       
        ╲╱                 i = 0                      

    ⎡1⎤
    ⎢ ⎥
    ⎢2⎥
    ⎢ ⎥
    ⎢3⎥
F = ⎢ ⎥
    ⎢4⎥
    ⎢ ⎥
    ⎢5⎥
    ⎢ ⎥
    ⎣6⎦

    ⎡mᵢ⎤
    ⎢  ⎥
    ⎢mᵢ⎥
    ⎢  ⎥
    ⎢mᵢ⎥
M = ⎢  ⎥
    ⎢mᵢ⎥
    ⎢  ⎥
    ⎢mᵢ⎥
    ⎢  ⎥
    ⎣mᵢ⎦

    1
T = ─
    f

Dabei entspricht $\mathbf{u}$ dem Verschiebungsvektor infolge des Kraftvektors $\mathbf{F}$. Der Verschiebungsvektor kann mittels Nachgiebigkeitsmatrix bestimmt werden.

#### Nachgiebigkeitsmatrix

Für gleichbleibende Geschosshöhen und Geschosssteifigkeiten lässt sich die Nachgiebigkeitsmatrix leicht mittels dem Bildungsgesetz in @eq-ekv_6_bildungsgesetz ermitteln.

$$
\hat{f}_{i,j} = \frac{H^3}{6EI} \cdot j^2(3i-j) \text{ für } i\geq j
$${#eq-ekv_6_bildungsgesetz}

$\hat{\mathbf{f}}$ entspricht der Nachgiebigkeitsmatrix mit den Einträgen $\hat{f}_{i,j}$.

Beachte dabei dass die @eq-ekv_6_bildungsgesetz nur für $i\geq j$ gilt. Die Einträge entsprechen folgendem Schema:

In [10]:
display(nachgiebigkeitsmatrix_nach_bildungsgesetz(n)[1])

⎡f_(1, 1)  f_(1, 2)  f_(1, 3)  f_(1, 4)  f_(1, 5)  f_(1, 6)⎤
⎢                                                          ⎥
⎢f_(2, 1)  f_(2, 2)  f_(2, 3)  f_(2, 4)  f_(2, 5)  f_(2, 6)⎥
⎢                                                          ⎥
⎢f_(3, 1)  f_(3, 2)  f_(3, 3)  f_(3, 4)  f_(3, 5)  f_(3, 6)⎥
⎢                                                          ⎥
⎢f_(4, 1)  f_(4, 2)  f_(4, 3)  f_(4, 4)  f_(4, 5)  f_(4, 6)⎥
⎢                                                          ⎥
⎢f_(5, 1)  f_(5, 2)  f_(5, 3)  f_(5, 4)  f_(5, 5)  f_(5, 6)⎥
⎢                                                          ⎥
⎣f_(6, 1)  f_(6, 2)  f_(6, 3)  f_(6, 4)  f_(6, 5)  f_(6, 6)⎦

Unter strikter Anwendung von @eq-ekv_6_bildungsgesetz folgt daraus:

In [11]:
display(nachgiebigkeitsmatrix_nach_bildungsgesetz(n)[2])

⎡2   0    0    0    0    0 ⎤
⎢                          ⎥
⎢5   16   0    0    0    0 ⎥
⎢                          ⎥
⎢8   28  54    0    0    0 ⎥
⎢                          ⎥
⎢11  40  81   128   0    0 ⎥
⎢                          ⎥
⎢14  52  108  176  250   0 ⎥
⎢                          ⎥
⎣17  64  135  224  325  432⎦

Aufgrund von Symmetrie kann diese abschliessend über die Diagonale gespiegelt werden:

In [12]:
display(sp.Eq(f_hat/sp.UnevaluatedExpr(H_i**3 /(6*E*I)), eq_f_hat.rhs/sp.UnevaluatedExpr(H_i**3 /(6*E*I))))

                ⎡2   5    8   11   14   17 ⎤
                ⎢                          ⎥
                ⎢5   16  28   40   52   64 ⎥
                ⎢                          ⎥
6⋅E⋅I           ⎢8   28  54   81   108  135⎥
─────⋅\hat{f} = ⎢                          ⎥
   3            ⎢11  40  81   128  176  224⎥
 Hᵢ             ⎢                          ⎥
                ⎢14  52  108  176  250  325⎥
                ⎢                          ⎥
                ⎣17  64  135  224  325  432⎦

Durch Multiplikation der *Nachgiebigkeit* mit der *Einwirkung* resultiert die Deformation.

In [13]:
display(eq_u, sh.eq_subs(eq_u,eq_f_hat, eq_F), sh.eq_subs(eq_u, eq_f_hat, eq_F).doit())

u = \hat{f}⋅F

    ⎡     3         3          3          3          3          3  ⎤    
    ⎢   Hᵢ        Hᵢ         Hᵢ         Hᵢ         Hᵢ         Hᵢ   ⎥    
    ⎢2⋅─────   5⋅─────    8⋅─────   11⋅─────   14⋅─────   17⋅───── ⎥    
    ⎢  6⋅E⋅I     6⋅E⋅I      6⋅E⋅I      6⋅E⋅I      6⋅E⋅I      6⋅E⋅I ⎥    
    ⎢                                                              ⎥    
    ⎢     3          3         3          3          3          3  ⎥    
    ⎢   Hᵢ         Hᵢ        Hᵢ         Hᵢ         Hᵢ         Hᵢ   ⎥    
    ⎢5⋅─────   16⋅─────  28⋅─────   40⋅─────   52⋅─────   64⋅───── ⎥    
    ⎢  6⋅E⋅I      6⋅E⋅I     6⋅E⋅I      6⋅E⋅I      6⋅E⋅I      6⋅E⋅I ⎥    
    ⎢                                                              ⎥ ⎡1⎤
    ⎢     3          3         3          3           3          3 ⎥ ⎢ ⎥
    ⎢   Hᵢ         Hᵢ        Hᵢ         Hᵢ          Hᵢ         Hᵢ  ⎥ ⎢2⎥
    ⎢8⋅─────   28⋅─────  54⋅─────   81⋅─────   108⋅─────  135⋅─────⎥ ⎢ ⎥
    ⎢  6⋅E⋅I      6⋅E⋅I     6⋅E⋅I      6⋅E⋅I       

    ⎡      3 ⎤
    ⎢ 42⋅Hᵢ  ⎥
    ⎢ ────── ⎥
    ⎢  E⋅I   ⎥
    ⎢        ⎥
    ⎢      3 ⎥
    ⎢925⋅Hᵢ  ⎥
    ⎢─────── ⎥
    ⎢ 6⋅E⋅I  ⎥
    ⎢        ⎥
    ⎢      3 ⎥
    ⎢950⋅Hᵢ  ⎥
    ⎢─────── ⎥
    ⎢ 3⋅E⋅I  ⎥
u = ⎢        ⎥
    ⎢       3⎥
    ⎢1535⋅Hᵢ ⎥
    ⎢────────⎥
    ⎢ 3⋅E⋅I  ⎥
    ⎢        ⎥
    ⎢       3⎥
    ⎢2173⋅Hᵢ ⎥
    ⎢────────⎥
    ⎢ 3⋅E⋅I  ⎥
    ⎢        ⎥
    ⎢       3⎥
    ⎢5663⋅Hᵢ ⎥
    ⎢────────⎥
    ⎣ 6⋅E⋅I  ⎦

Durch einsetzen der bestimmten Verformung in die Gleichung der Eigenfrequenz folgt:

In [14]:
eq_f_subs = sh.eq_subs(eq_f, eq_u, eq_M, eq_f_hat, eq_F).doit()
eq_T_subs = sh.eq_subs(eq_T, eq_f_subs).doit()

display(eq_f_subs, eq_T_subs)

                      ________
                     ╱  E⋅I   
    √43665341610⋅   ╱  ────── 
                   ╱     3    
                 ╲╱    Hᵢ ⋅mᵢ 
f = ──────────────────────────
            4993178⋅π         

      √43665341610⋅π  
T = ──────────────────
              ________
             ╱  E⋅I   
    8745⋅   ╱  ────── 
           ╱     3    
         ╲╱    Hᵢ ⋅mᵢ 

### Grundschwingzeit

#### X-Richtung

Es gilt $I$ mit $I_y$ zu substituieren.

In [15]:
eq_u_x = sh.eq_subs(eq_u).subs(I, I_y).subs(u, u_x).doit()

eq_f_x = eq_f_subs.subs(I, I_y).subs(f, f_x)
eq_T_x = eq_T_subs.subs(I,I_y).subs(T, T_x)


display(eq_u_x, eq_f_x.subs(params).evalf(3), eq_T_x.subs(params).evalf(3))


uₓ = \hat{f}⋅F

      1.88 
fₓ = ──────
     second

Tₓ = 0.531⋅second

#### Y-Richtung

Es gilt $I$ mit $I_x$ zu substituieren.

In [16]:

eq_u_y = sh.eq_subs(eq_u,eq_f_hat, eq_F).subs(I, I_x).subs(u, u_y).doit()
eq_f_y = eq_f_subs.subs(I, I_x).subs(f, f_y)
eq_T_y = eq_T_subs.subs(I,I_x).subs(T, T_y)

display(eq_u_y, eq_f_y.subs(params).evalf(3), eq_T_y.subs(params).evalf(3))

      ⎡      3 ⎤
      ⎢ 42⋅Hᵢ  ⎥
      ⎢ ────── ⎥
      ⎢  E⋅Iₓ  ⎥
      ⎢        ⎥
      ⎢      3 ⎥
      ⎢925⋅Hᵢ  ⎥
      ⎢─────── ⎥
      ⎢ 6⋅E⋅Iₓ ⎥
      ⎢        ⎥
      ⎢      3 ⎥
      ⎢950⋅Hᵢ  ⎥
      ⎢─────── ⎥
      ⎢ 3⋅E⋅Iₓ ⎥
u_y = ⎢        ⎥
      ⎢       3⎥
      ⎢1535⋅Hᵢ ⎥
      ⎢────────⎥
      ⎢ 3⋅E⋅Iₓ ⎥
      ⎢        ⎥
      ⎢       3⎥
      ⎢2173⋅Hᵢ ⎥
      ⎢────────⎥
      ⎢ 3⋅E⋅Iₓ ⎥
      ⎢        ⎥
      ⎢       3⎥
      ⎢5663⋅Hᵢ ⎥
      ⎢────────⎥
      ⎣ 6⋅E⋅Iₓ ⎦

       1.37 
f_y = ──────
      second

T_y = 0.732⋅second

### Abminderung der Steifigkeit

Um die Rissbildung zu berücksichtigen wird die Steifigkeit auf 30 % abgemindert.

#### X-Richtung

In [17]:
eq_f_x_red = eq_f_x.subs(E,0.3*E)
eq_T_x_red = eq_T_x.subs(E,0.3*E)

display(eq_f_x_red.subs(params).evalf(3), eq_T_x_red.subs(params).evalf(3))

      1.03 
fₓ = ──────
     second

Tₓ = 0.97⋅second

#### Y-Richtung

In [18]:
eq_f_y_red = eq_f_y.subs(E,0.3*E)
eq_T_y_red = eq_T_y.subs(E,0.3*E)

display(eq_f_y_red.subs(params).evalf(3), eq_T_y_red.subs(params).evalf(3))

      0.748 
f_y = ──────
      second

T_y = 1.34⋅second